# PRISM precipitation climatology pre-process

In [1]:
import os
import sys
import time
import h5py
import pygrib

import numpy as np
from glob import glob

from datetime import datetime, timedelta

In [16]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
filename = '/glade/campaign/cisl/aiml/ksha/PRISM/PRISM_ppt_30yr_normal_4kmM4_{:02d}_asc/*asc.asc'

# prism size info (for PRSIM website)
prism_x = 621
prism_y = 1405

In [38]:
PRISM = np.empty((12, prism_x, prism_y))

for mon in range(12):

    prism_name = glob(filename.format(mon+1))[0]
    
    with open(prism_name, 'r') as prism_io:
        prism_header = prism_io.readlines()[:6]
     
    # Read the PRISM ASCII raster header
    prism_header = [item.strip().split()[-1] for item in prism_header]
    prism_cols = int(prism_header[0])
    prism_rows = int(prism_header[1])
    prism_xll = float(prism_header[2])
    prism_yll = float(prism_header[3])
    prism_cs = float(prism_header[4])
    prism_nodata = float(prism_header[5])
    
    # Read in the PRISM array
    prism_array = np.loadtxt(prism_name, dtype=float, skiprows=6)
    # Set the nodata values to nan
    prism_array[prism_array == prism_nodata] = np.nan
    # PRISM data is stored as an integer but scaled by 100
    prism_array *= 0.01
    PRISM[mon, ...] = prism_array
    
    # prepare lat/lon information once only
    if mon == 0:
    
        prism_extent = [
            prism_xll, prism_xll + prism_cols * prism_cs,
            prism_yll, prism_yll + prism_rows * prism_cs]
        
        # Build arrays of the lat/lon points for each cell
        lons = np.arange(prism_extent[0], prism_extent[1], prism_cs)
        lats = np.arange(prism_extent[3], prism_extent[2], -prism_cs)
        lons, lats = np.meshgrid(lons, lats)

## Interpolate PRISM to 0.1 deg

In [21]:
from scipy.interpolate import griddata
import scipy.interpolate as spint
from scipy.spatial import Delaunay
import itertools

def interp_weights(xy, uv, d=2):
    tri = Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

In [23]:
with h5py.File(save_dir+'CNN_domain.hdf', 'r') as h5io:
    lon_01 = h5io['lon_01'][...]
    lat_01 = h5io['lat_01'][...]

In [25]:
# Computed once and for all
vtx, wts = interp_weights(np.vstack([lons.ravel(), lats.ravel()]).T, 
                          np.vstack([lon_01.ravel(), lat_01.ravel()]).T)

In [31]:
PRISM_01 = np.empty((12,)+lon_01.shape)

for mon in range(12):
    prism_ = PRISM[mon, ...]

    prism_temp = interpolate(prism_.ravel(), vtx, wts)
    prism_temp = prism_temp.reshape(lon_01.shape)
    prism_temp[prism_temp<0] = 0
    prism_temp[250:, :] = np.nan
    PRISM_01[mon, ...] = prism_temp

In [43]:
# tuple_save = (lons, lats, PRISM, PRISM_01)
# label_save = ['lon_4km', 'lat_4km', 'PRISM_4km', 'PRISM_01']
# du.save_hdf5(tuple_save, label_save, save_dir, 'PRISM_Climatology.hdf')

In [44]:
# plt.pcolormesh(PRISM_01[2, 64:196, 64:196], cmap=plt.cm.nipy_spectral_r)